This notebook checks the availability of the IRAC/WISE files for the stellar profilies for models 3&4.

In [1]:
#import packages and initialize settings
import os
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.io import ascii
from reproject import reproject_interp

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']="white"

import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
from SNeGenerationModels import runModels
from FindNearestMC import nearestMCMethod

In [2]:
# Get SNeCO information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")

database[0:1]

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str80
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/ESO097-013_alphaCO21_scaling.fits


In [3]:
galaxies = np.array(database["galaxy"], dtype='object')   
SNe, SNra, SNdec  = np.array(database["SN_name"], dtype='object'), np.array(database["SN_ra"], dtype='object'), np.array(database["SN_dec"], dtype='object')
telOrient = np.array(database["telOrient"], dtype='object')
centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))] 

#fileName = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc0628_irac1_1kpc.fits"
fileName = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc0628_irac1_gauss7p5.fits"
hdulist = pyfits.open(fileName)
map = hdulist[0].data
#wcs = WCS(hdulist[0].header, naxis=2)


In [4]:
# generate list of galaxies in IRAC sample and Phangs Alma coverage maps
# "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "_12m+7m+tp_co21_150pc_coverage2d.fits"

IRFileList, PACovFileList = [],[]

for i in range(len(galaxies)):
    PACovFileList.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxies[i] + "/" + galaxies[i] + "_" + telOrient[i] + "_co21_150pc_coverage2d.fits")

    if galaxies[i] == "circinus":
        galaxies[i] = "eso097-013"
    fileCheck = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/" + galaxies[i] + "_irac1_gauss7p5.fits"
    
    if os.path.isfile(fileCheck) == False:
        fileCheck = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/" + galaxies[i] + "_w1_gauss7p5.fits"
        if os.path.isfile(fileCheck) == False:
            fileCheck = "No IRAC or WISE File for " + galaxies[i] + "." 
        else:
            print(galaxies[i], "WISE")
    else: 
        print(galaxies[i],"IRAC")
    IRFileList.append(fileCheck)

    

eso097-013 WISE
ngc0253 IRAC
ngc0628 IRAC
ngc1068 IRAC
ngc1087 IRAC
ngc1097 IRAC
ngc1097 IRAC
ngc1365 IRAC
ngc1365 IRAC
ngc1365 IRAC
ngc1433 IRAC
ngc1511 IRAC
ngc1559 IRAC
ngc1559 IRAC
ngc1559 IRAC
ngc1559 IRAC
ngc1566 IRAC
ngc1566 IRAC
ngc1637 IRAC
ngc1672 IRAC
ngc2775 IRAC
ngc2997 WISE
ngc2997 WISE
ngc3239 IRAC
ngc3627 IRAC
ngc3627 IRAC
ngc3627 IRAC
ngc3627 IRAC
ngc4254 IRAC
ngc4254 IRAC
ngc4254 IRAC
ngc4254 IRAC
ngc4303 IRAC
ngc4303 IRAC
ngc4303 IRAC
ngc4303 IRAC
ngc4303 IRAC
ngc4303 IRAC
ngc4303 IRAC
ngc4321 IRAC
ngc4321 IRAC
ngc4321 IRAC
ngc4321 IRAC
ngc4321 IRAC
ngc4321 IRAC
ngc4424 IRAC
ngc4457 IRAC
ngc4496a IRAC
ngc4496a IRAC
ngc4536 IRAC
ngc4579 IRAC
ngc4579 IRAC
ngc4945 WISE
ngc5128 WISE
ngc5128 WISE
ngc5236 IRAC
ngc5236 IRAC
ngc5236 IRAC
ngc5236 IRAC
ngc5236 IRAC
ngc5530 WISE
ngc6744 WISE
ngc7793 IRAC


In [6]:
# for i in range(len(fileList)):
#     print(fileList[i])

database.add_column(PACovFileList, name ="PACovFile")
database.add_column(IRFileList, name = "IFRFile")

database[0:1]

ascii.write(database, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  

Missing IRAC files for Circinus, NGC2997, NGC4945, NGC5128, NGC5530, NGC6744
Wise files available in ZOMGS directory
Missing 150pc res and coverage files for NGC 1068, 1672, & 4579
for now will leave out and maybe revisit with another resolution later?

What resolution is this map: 7p5 arcsec
W1 wise is 3.4 microns.
"IRAC is a four-channel camera that provides simultaneous 5.2 x 5.2 arcmin images at 3.6, 4.5, 5.8, and 8 microns (the black instrument in the lower right part of the above image; see the optical housing model and the conceptual layout diagram. The pixel size is 1.22 arcsec in all bands. "

In [ ]:
# remove duplicate galaxies, grouping SNe together in their own lists.

gal_group = database.group_by("galaxy")

SNr, SNd, SNa = [],[],[]
gala, ifrs, covs, imgs, errs, alco, cocs, pans, incl, gdis, SNrs, SNdc, SNea = [],[],[],[],[],[],[],[],[],[],[],[],[]

for i in range(len(database)):
    gal = database["galaxy"][i]
    ifr = database["IFRFile"][i]
    cov = database["PACovFile"][i]
    img = database["ImageFile150"][i]
    ers = database["ErrFile150"][i]
    aco = database["AlphaCOFile"][i]
    ccs = centerCoords[i]
    pas = database["orient_posang"][i]
    inc = database["orient_incl"][i]
    gds = database["dist"][i]
    SNr.append(SNra[i])
    SNd.append(SNdec[i])
    SNa.append(SNe[i])
    
    if (i+1) in gal_group.groups.indices:
        #print(gal)
        if gal != "ngc1068" and gal != "ngc1672" and gal != "ngc4579":
            #print(gal)
            gala.append(gal)
            ifrs.append(ifr)
            covs.append(cov)
            imgs.append(img)
            errs.append(ers)
            alco.append(aco)
            cocs.append(ccs)
            pans.append(pas)
            incl.append(inc)
            gdis.append(gds)
            SNrs.append(SNr)
            SNdc.append(SNd)
            SNea.append(SNa) 
            SNr, SNd, SNa = [],[],[]
        else: pass
    else: pass